# Linked SV analysis for SVclone paper

Please install sv_tools before running:

    git clone git@github.com:mcmero/sv_tools.git
    cd sv_tools
    python setup.py install
    
Run the `analyse_select_pcawg.RmD` R markdown notebook *before* running this notebook.

In [1]:
import pandas as pd
import numpy as np
import statsmodels
import statsmodels.stats.multitest as smm
import contextlib
import io
import sys
import os
import itertools
from sv_tools import sv_data, sv_diagram, kc_tests
from os.path import expanduser 

In [5]:
consensus_pp = '~/data/pcawg_data/consensus.20170217.purity.ploidy.txt'
cc_path = '~/data/svclone_paper_20181219/%s/ccube_out/%s_cluster_certainty.txt'
bal_enr_samples = 'pcawg_bal_enr/bal_enr_samples.txt'
other_samples = 'pcawg_bal_enr/other_samples.txt'

In [6]:
@contextlib.contextmanager
def nostdout():
    save_stdout = sys.stdout
    sys.stdout = io.BytesIO()
    yield
    sys.stdout = save_stdout

purities = pd.read_csv(expanduser(consensus_pp), delimiter='\t')
chroms = [str(x) for x in np.array(range(1,23))] + ['X', 'Y']
aliquots = pd.read_csv(bal_enr_samples, header=-1)
aliquots = aliquots[0].values
aliquots_other = pd.read_csv(other_samples, header=-1)
aliquots_other = aliquots_other[0].values

In [7]:
# generate SV files as input for sv_tools
for aliquot_id in np.append(aliquots, aliquots_other):
    sv_file = expanduser(cc_path % (aliquot_id, aliquot_id))
    svs = pd.read_csv(sv_file, delimiter='\t')

    if len(svs) > 0:
        svs['average_proportion'] = svs[['average_proportion1','average_proportion2']].apply(np.mean, axis=1)
        pur = purities[purities.samplename == aliquot_id].purity.values[0]
        ccfs = svs.average_proportion / pur
        
        svs = svs[['chr1','pos1','dir1','chr2','pos2','dir2']]            
        rename_cols =  {'chr1': 'chrom1', 'dir1': 'strand1', 'chr2': 'chrom2', 'dir2': 'strand2'}
        svs = svs.rename(columns = rename_cols)

        sv_file = 'pcawg_bal_enr/%s_svs.txt' % aliquot_id
        svs.to_csv(sv_file, sep='\t', index=False)
                
        sc_svs = svs[ccfs < 0.7]
        sc_sv_file = 'pcawg_bal_enr/%s_sc_svs.txt' % aliquot_id
        sc_svs.to_csv(sc_sv_file, sep='\t', index=False)

In [8]:
result_cols = ['aliquot', 'svs', 'cl_svs', 'sc_svs', 'sig_clus', 
               'clonal_sig_svs', 'subclonal_sig_svs', 'chroms_tested', 
               'chroms_walked', 'linked_chroms']
chrom_cols = ['chrom', 'all_svs_tested', 'cl_svs_tested', 'sc_svs_tested', 
                 'ks_pval', 'cl_ks_pval', 'sc_ks_pval', 'walk_pval']

def test_sv_linkage(aliquots):

    test_results = pd.DataFrame(columns=result_cols)
    for aliquot_id in aliquots:
        sv_file = 'pcawg_bal_enr/%s_svs.txt' % aliquot_id
        sc_sv_file = 'pcawg_bal_enr/%s_sc_svs.txt' % aliquot_id

        if not os.path.exists(sv_file) or not os.path.exists(sc_sv_file):
            continue

        svs      = pd.read_csv(sv_file, delimiter='\t')
        svs      = svs[svs.chrom1 == svs.chrom2]
        sc_svs   = pd.read_csv(sc_sv_file, delimiter='\t')
        itrx_svs = sc_svs[['chrom1','chrom2']]
        itrx_svs = itrx_svs[itrx_svs.chrom1.map(str) != itrx_svs.chrom2.map(str)]
        itrx_svs = itrx_svs.drop_duplicates()
        if len(svs) == 0 or len(sc_svs) == 0: continue

        print(aliquot_id)
        chrom_results = pd.DataFrame()
        chroms_tested = 0
        for chrom in chroms:

            bps = sv_data.get_breakpoints(sv_file, chrom)
            sc_bps = sv_data.get_breakpoints(sc_sv_file, chrom)
            cl_bps = [bp for bp in bps if bp not in sc_bps]

            fusions = sv_data.get_fusions(sv_file, chrom)
            sc_fusions = sv_data.get_fusions(sc_sv_file, chrom)
            cl_fusions = [fus for fus in fusions if fus not in sc_fusions]

            if len(sc_bps) < 4 or len(cl_bps) < 4:
                continue
            chroms_tested += 1

            walk=''.join(pd.Series(map(lambda x: x.type(), fusions)).values)
            if walk.count('H') > 0 and walk.count('T') > 0:
                with nostdout():
                    kct = kc_tests.F_walk(walk)
            else:
                kct = [float('nan'), float('nan'), float('nan')]

            with nostdout():
                sc_ks_test = kc_tests.test_A(sc_bps)
                cl_ks_test = kc_tests.test_A(cl_bps)
                ks_test    = kc_tests.test_A(bps)

            results = pd.DataFrame([[chrom, len(bps)/2, len(sc_bps)/2, len(cl_bps)/2, ks_test.pvalue, 
                                     cl_ks_test.pvalue, sc_ks_test.pvalue, kct[2]]], columns = chrom_cols)
            chrom_results = chrom_results.append(results)

        if len(chrom_results) > 0:            
            sig_ks    = smm.multipletests(chrom_results.ks_pval.values, method='fdr_bh')[0]
            sc_sig_ks = smm.multipletests(chrom_results.sc_ks_pval.values, method='fdr_bh')[0]
            cl_sig_ks = smm.multipletests(chrom_results.cl_ks_pval.values, method='fdr_bh')[0]
            sig_walk  = np.sum(smm.multipletests(chrom_results.walk_pval.values, method='fdr_bh')[0])            
            chroms_tested = len(chrom_results)
            
            # check linkage of chroms with clusters
            linked_chroms = 0
            if len(itrx_svs) > 0 and len(chrom_results) > 0:
                sig_scs = chrom_results[np.array(sc_sig_ks)]
                perms = np.array([c for c in itertools.permutations(sig_scs.chrom, 2)])
                if len(perms) > 0:
                    itx_chr1 = itrx_svs.chrom1.map(str).values
                    itx_chr2 = itrx_svs.chrom2.map(str).values
                    matched = []
                    for perm in perms:
                        match_chrom1 = np.array(itx_chr1 == str(perm[0]))
                        match_chrom2 = np.array(itx_chr2 == str(perm[1]))
                        n_matched    = np.sum(np.logical_and(match_chrom1, match_chrom2))
                        matched.append(n_matched > 0)
                    linked_chroms = np.sum(np.array(matched))

            sig_ks    = sum(chrom_results[sig_ks].all_svs_tested)
            sc_sig_ks = sum(chrom_results[sc_sig_ks].sc_svs_tested)
            cl_sig_ks = sum(chrom_results[cl_sig_ks].cl_svs_tested)
            
            results   = pd.DataFrame([[aliquot_id, len(svs), len(svs) - len(sc_svs), 
                                       len(sc_svs), sig_ks, cl_sig_ks, sc_sig_ks, 
                                       chroms_tested, sig_walk, linked_chroms]], columns=result_cols)
            test_results = test_results.append(results)
    return test_results

In [9]:
bal_enr_linkage = test_sv_linkage(aliquots)
other_linkage = test_sv_linkage(aliquots_other)

00c27940-c623-11e3-bf01-24c6515278c0
00db4dc2-3ec7-4ff9-9233-d69c8c8a607f
01df36af-3617-40fc-9892-f54ce433cf71
07d20658-3db4-47e7-877b-66536266edfc
0831e45e-c623-11e3-bf01-24c6515278c0
0ab4d782-9a50-48b9-96e4-6ce42b2ea034
0bfd1043-8181-e3e4-e050-11ac0c4860c5
0bfd1068-3fd8-a95b-e050-11ac0c4860c3
1021b60d-f7b2-43b0-b2cc-f282d619d533
105a51c4-cc7e-4d0f-9cf8-e4d64a31d14d
127b0f7d-d24e-48b7-ac25-d3f14a43952d
12f038e1-00af-4c64-a2e0-9e63323492ef
134c9a92-e91e-4347-a9c4-727279edebb1
15e7d981-8c27-4b2b-b4f8-626e22021895
15f90ef0-831b-40a3-98bd-ec226a9e8b26
16df7888-2480-4394-8856-d57a6ef371d2
19c1c97f-a3ec-44a8-8a20-6f97caed1a4f
1ac15380-04a2-42dd-8ade-28556a570e80
1aff91a6-1b0f-4575-8f4b-4e064a50b886
1c28e44a-6e6c-44ed-b58a-e3262c0e6759
1fff8b62-534b-4d71-a65f-e5f93b8b50ed
2290b078-6a5b-4c83-9dfb-b525bbf14e4e
25c76a8f-77c0-4650-bddf-45ed0c10a2e6
25e20393-752b-4796-9001-0e22ee04c586
278b2498-1d64-493b-ac43-3489ec86f313
295aac88-c623-11e3-bf01-24c6515278c0
2b40a733-7a63-4bb8-a953-95a4ee28f962
2

2009e5e7-1796-445b-8677-46b3804fe0bf
207f8a42-5b05-4876-b0ae-ebfaeea27844
20d1b88b-3ff6-4201-a748-6a993500c652
20e02396-e676-412d-9724-44a428919cdb
2660825c-68f0-4631-948e-6da158edbe9e
27b56bf2-7a9d-4061-98d4-61fe2761578c
28f41a20-b6d6-4ecc-888f-72f779ad9af7
29578759-9ccb-4a24-b5b8-c45ebd4339d9
29a00d78-b9bb-4c6b-b142-d5b8bfa63455
2b000af3-2c9d-4eaa-af3f-8101b7425c37
2b78de4e-4c8b-4adf-a058-3dae797e7881
2bf5b018-9f19-4fbd-9e1f-7d958aabe5d1
2c48eefe-2a08-47bf-8e4d-cbaab6777150
2c6f1862-bb82-4e7e-9cb3-338bdf022ff4
2c9dc04b-e9ec-4cf1-ab2c-a18edb30dd37
2e43e0ca-54ea-482e-acf2-0048d9187a5c
3166f1ae-678d-42ae-9d44-d3d25d6860c7
31c75873-abb4-4d88-9e2f-07497a6c892d
3232f77f-b745-4232-a802-6699b6356efd
33bf46dd-16b3-49c6-80d7-76caf27aa0f4
34ab4c57-5240-4af7-a329-a5ab55934fd4
351db483-a70e-496d-b70a-7449875121ee
352fbbb4-88a5-4354-b1fa-3a01da3fbfa7
3585e133-b3c1-4d90-b5f2-2b867e0ae0ec
36d1a85e-a09b-4537-86e0-eaf1eb03aed8
3728982b-4547-4249-bd42-72a91d3fda8c
374cbd87-428e-4509-85c1-b7d3302c30a0
3

ece0f3a4-a204-4c52-bb1f-88d44a875b2e
ecf4e05a-0912-4b93-ad66-323002f0c845
ed46bb2e-97e3-4914-a086-de80e00d6ee8
ed4b53cb-5473-4bc1-9d84-0b0ad93cbd57
ed7be9ae-e603-4731-8d91-a8285abaee33
edac1323-2497-45e6-9148-e9c955292ba2
ef673d3d-2031-4036-ba25-4bc7ef04075b
f1504811-8363-41e6-b43c-62452b1262d3
f37971bd-ec65-4840-8d4f-678692cee695
f48c3c82-bebe-4b8e-909e-e1a51a7142ec
f4ada7a2-c4ac-4f89-ada7-4645861002cb
f6114c69-71a1-47d5-9b28-b0227b1872f7
f658c350-fb89-4268-8a59-a07e365f4221
f81693ba-09ee-4201-a389-0ceeda8a4636
f848b66f-bd9e-4fba-afd4-eb58848d1ef4
f8f749b7-547d-49fa-9da2-44eed962b6fd
f98de26b-c7d6-435d-81fa-1f1869da9087
f9a81200-5381-496a-8062-099f9e793618
f9c0a08b-d1e5-4c18-e040-11ac0c4864df
f9c1df10-25b8-c8ae-e040-11ac0d486375
f9c23ce8-7f1c-9417-e040-11ac0d482562
f9c39eb7-39a9-6626-e040-11ac0d4870c2
f9c3eaad-a0d9-8bf8-e040-11ac0d481d8e
f9c4e06c-e8a6-613b-e040-11ac0d4828ba
f9c51617-3fcb-91c5-e040-11ac0d484abf
f9c650e7-9053-78eb-e040-11ac0d4874bb
f9c65e3d-f3f7-dd5f-e040-11ac0d487b1f
f

In [16]:
n_svs         = float(sum(bal_enr_linkage.svs.values))
svs_linked    = sum(bal_enr_linkage.sig_clus.values) / n_svs
cl_svs_linked = sum(bal_enr_linkage.clonal_sig_svs.values) / n_svs
sc_svs_linked = sum(bal_enr_linkage.subclonal_sig_svs.values) / n_svs
perc_chrs_wlk = np.mean(bal_enr_linkage.chroms_walked.values / bal_enr_linkage.chroms_tested.values)
perc_chrs_lnk = np.sum(bal_enr_linkage.linked_chroms.values > 0) / float(len(bal_enr_linkage))

collated_cols = ['prop_svs_linked', 'prop_clonal_svs_linked', 'prop_subcl_svs_linked', 'frac_chrs_walkable', 'chroms_linked']
sv_linkage    = pd.DataFrame([svs_linked, cl_svs_linked, sc_svs_linked, perc_chrs_wlk, perc_chrs_lnk], index = collated_cols)

n_svs         = float(sum(other_linkage.svs.values))
svs_linked    = sum(other_linkage.sig_clus.values) / n_svs
cl_svs_linked = sum(other_linkage.clonal_sig_svs.values) / n_svs
sc_svs_linked = sum(other_linkage.subclonal_sig_svs.values) / n_svs
perc_chrs_wlk = np.mean(other_linkage.chroms_walked.values / other_linkage.chroms_tested.values)
perc_chrs_lnk = np.sum(other_linkage.linked_chroms.values > 0) / float(len(other_linkage))

sv_linkage    = pd.concat([sv_linkage, pd.DataFrame([svs_linked, cl_svs_linked, sc_svs_linked, 
                                                     perc_chrs_wlk, perc_chrs_lnk], index = collated_cols)], axis=1)

sv_linkage.columns = ['bal_enriched', 'other']
print(sv_linkage)   

                        bal_enriched     other
prop_svs_linked             0.970471  0.871229
prop_clonal_svs_linked      0.269430  0.164889
prop_subcl_svs_linked       0.391145  0.313271
frac_chrs_walkable          0.027340  0.020941
chroms_linked               0.503106  0.215909


In [17]:
import scipy.stats as stats

fracs_scnr_linked = bal_enr_linkage.subclonal_sig_svs.values / bal_enr_linkage.svs.values
fracs_other_linked = other_linkage.subclonal_sig_svs.values / other_linkage.svs.values

stats.ttest_ind(fracs_scnr_linked, fracs_other_linked)

Ttest_indResult(statistic=3.1028207134779766, pvalue=0.0020799283555914708)

In [18]:
scnr_prop_chroms_walked = bal_enr_linkage.chroms_walked.values / bal_enr_linkage.chroms_tested.values
other_prop_chroms_walked = other_linkage.chroms_walked.values / other_linkage.chroms_tested.values

stats.ttest_ind(scnr_prop_chroms_walked, other_prop_chroms_walked)

Ttest_indResult(statistic=0.8375608993167786, pvalue=0.40287449450568713)